# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [37]:
 #Imports
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.appName('logR').getOrCreate()

In [3]:
df = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [6]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [7]:
df.columns


['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
                            outputCol='features')

In [17]:
log_reg = LogisticRegression(featuresCol='features', labelCol='Churn')

In [38]:
#Using a short pipeline
Pipeline = Pipeline(stages = [assembler, log_reg])

In [41]:
train_set, test_set = df.randomSplit([0.7, 0.3])

In [42]:
model = Pipeline.fit(train_set)

In [43]:
results = model.transform(test_set)

In [63]:
results.select('Company', 'prediction').show()

+--------------------+----------+
|             Company|prediction|
+--------------------+----------+
|Steele, Bates and...|       0.0|
|      Thompson Group|       1.0|
|     Harvey-Mcdowell|       0.0|
|Smith, Harris and...|       0.0|
|   Campbell and Sons|       0.0|
|             Lee Ltd|       0.0|
|         Hoffman Ltd|       0.0|
|Green, Jackson an...|       0.0|
|           Evans LLC|       1.0|
|        Jones-Fisher|       0.0|
|            Gray Inc|       1.0|
|Lee, Daniels and ...|       0.0|
|      Johnson-Nelson|       0.0|
|      Woodward-Bruce|       0.0|
|         Hancock LLC|       0.0|
|Miller, Gardner a...|       0.0|
|        Knight-Price|       0.0|
|           Lopez PLC|       1.0|
|Jones, Reyes and ...|       0.0|
|          Austin Ltd|       0.0|
+--------------------+----------+
only showing top 20 rows



Compute the AUC

In [45]:
model_evaluation = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [46]:
AUC = model_evaluation.evaluate(results)

In [47]:
AUC

0.772794630404463

**Predict churn rate on new data (which companies should be assign an account manager)**

In [48]:
new_data = spark.read.csv('new_customers.csv', header=True, inferSchema=True)

In [50]:
new_res = model.transform(new_data)

In [62]:
new_res.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

